### Notes on Lilac paper (PCS with Linear time prover, Log Verifier and Proof-Size)

#### Abstract Notes

PCS is  field agnostic and transparent

For a poly- nomial with $N$ coefficients, LiLAC achieves $O(N)$ prover time, $O(\log N)$ verifier time, and $O(\log N)$ proof size.

Achieved through an optimized polynomial commitment strategy and the recursive application of the tensor IOPP.

The field-agnostic LiLAC is compared against Brake- down (CRYPTO 2023), which is based on a tensor IOP and satisfies field-agnosticity.
In experiments conducted over a 128-bit field with a coefficient size of $2^{30}$, the field-agnostic LiLAC achieves a proof size that is 3.7× smaller and a verification speed that is 2.2× faster, while main- taining a similar proof generation time compared to Brakedown.

The field-specific LiLAC is evaluated against WHIR (ePrint 2024/1586), which is based on an FRI. 
With a 128-bit field and a coeffi- cient size of $2^30$, the field-specific LiLAC achieves a proof generation speed that is 2.8× faster, a proof size that is 27% smaller, and a verification speed that is 14% faster compared to WHIR.


#### Technical Overview Notes

The Tensor IOPP has the following steps: a) The coefficient of the polynomial $f$ is written into a coefficient matrix $C$.  
b) The rows of $C$ are encoded using the encoding matrix $G$. The encoded matrix is called $E$ and is committed to by the prover.  
c) We have two checks: proximity and consistency, both being the same.   
The only difference is that proximity requires a random point, whereas consistency requires the evaluation point.  
If the evaluation point is random then Diamond and Posen show that running consistency check is enough.  
We explain consistency check.  
d) Let $f$ be $\ell$ variate and $\mathbf{z} \in \mathbb{F}^\ell$ be the evaluation point and $y$ be the claimed value of $f(\ell)$.  
e) Let $\mathbf{u} = \otimes_{i=0}^{\ell-1} (1-z_i, z_i)$, and $\mathbf{u}_l$ and $\mathbf{u}_r$ be the left and the right halves of $\mathbf{u}.  
g) The prover first computes $\mathbf{u}_l \cdot C$ and sends it to the verifier.  
e) The verifier samples a set $I$ of size $t$ determined by the security parameter $\lambda$. The set is the subset of the column indices.
f) For every $i \in I$, the verifier checks the $i$ location of the encoding of $\mathbf{u}_l \cdot C$ and the $i$-th location of $\mathbf{u}_l \cdot E$. If both are the same then it proceeds else it rejects.  
g) Check $y$ = $\mathbf{u}_l \cdot C\cdot (\mathbf{u}_R)^T$.  

--------------------------

They perform the verifier checks using sum-check protocols.
In particular, let $\mathbf{r} = \mathbf{u}_l \cdot C$ we have the following two sum-checks that we need to do:
$$\sum_{i}\widetilde{\mathbf{r}}(i) \cdot \widetilde{G}(i,j) = \sum_{i}\widetilde{\mathbf{u}}_l(i) \cdot \widetilde{E}(i,j) $$
$$ \sum_{i} \widetilde{\mathbf{r}}(i)\cdot \widetilde{\mathbf{u}}_r(i) = y$$

However, directly applying this to the equations above would require evaluating the polynomials $\widetilde{G}$ and $\widetilde{E}$, which have more than $N = 2^\ell$ elements. This would negate the efficiency gains of using the sum-check protocol. 
To address this issue, we leverage the fact that tensor IOPP only requires verification at a limited number of random positions, specifically $t$, determined by the security parameter $\lambda$. 


Using this insight, we construct reduced matrices $G′$ and $E′$ that include only the elements corresponding to the selected positions in $G$ and $E$. The reduced components, $G′$ and $E′$, along with the vectors $\mathbf{r}$ and $\mathbf{u}_l$  are then combined to form the coefficient matrix $C′$ for use in the next tensor IOPP round. By executing the subsequent tensor IOPP on this reduced matrix, the protocol achieves significant efficiency improvements.

**Commitments to $G$ and $E$**: Initially, Merkle trees are constructed for both $G$ and $E$ to generate their respective commitments. Each matrix is first committed by constructing a Merkle tree over its column vectors. The roots of these column-level Merkle trees then act as intermediate nodes to build a second Merkle tree along the row direction. The root of this combined Merkle tree, denoted as $cm$, serves as the final commitment value for the corresponding matrix.

Intermediate nodes corresponding to the $t$ selected columns are extracted from the existing commitments $cm_G$ and $cm_E$. 
These nodes are then utilized to construct a new Merkle tree commitment for the subsequent recursive round. By reusing the previously computed commitments for row vectors and selectively extracting only the required intermediate nodes, the scheme minimizes the computational overhead involved in generating new commitments. This approach significantly reduces the cost of recomputing commitments in recursive rounds to $O(t)$.


In this process, the newly constructed reduced matrices $G′$ and $E′$ have dimensions reduced to $\sqrt{N} \times t$ compared to $\sqrt{N} \times \sqrt{N}$ the original matrices $G$ and $E$.


To verify the correctness of the reduced matrices and the associated encoding operations, all MLPCS evaluations for $G′$, $E′\in \mathbb{F}^{\sqrt{N}\times t}$ and $\mathbf{r}$ and $\mathbf{u}_l$ are combined into a single MLPCS. This combined MLPCS is then evaluated to ensure that the evaluations of $G′$, $E′$, $\mathbf{r}$ and $\mathbf{u}_l$ match as expected.
The newly constructed MLPCS, containing $2t N + 2 N$ coefficients, is then invoked for the next recursive round.







#### Security Error in The Protocol

The protocol commits to the coefficient matrix and only commits the encoded matrix only during the evaluation protocol.
This is crucially used to remove the $\lambda^2$ factor that we get in the protocol I discussed with Bhargav.
Essentially, they merklize the coefficient matrix columns and use the roots to build the final Merkle tree.
The encoded matrix has the property that the coefficient matrix is the first $\sqrt{N}$ columns, assuming the code is systematic.
During the evaluation protocol commits to the remaining $(1 - \rho)\sqrt{N}$ columns and sends it to the verifier.
The verifier uses the commit to the coefficient matrix and the above commitment to derive the commitment to the encoded matrix.

The prove and verifier then engage in the reduce protocol to reduce the size of the polynomial.
The commitment to the coefficient matrix of the new polynomial is dervied by the verifier using existing knowledge but not for the enocoding of it.
Here is where the protoocol breaksdown.

Suppose a malicious prover $P'$ creates a encoding using a coeffcieint matrix $C'$ that differs from $C$ in just one column, and creates an encoding $E'$.
$P'$ sends the commitment to the remaining parts of $E'$ instead of $E$ in the above process.
The proximity check will still pass with high probability.
Hence this is not secure!


### Notes on HyperNova

#### Introduction Notes

The paper primarily builds on the previous work Nova.
Nova's computational modes proves incrmental computation where each step executes a non-deterministic circuit.
To prove such computations Nova uses a folding scheme for an NP-complete language to recursively tranform the task of proving $N$ steps of a computation into the task of proving a single step of the computation.
It then applies ZK-SNARK to prove the single step obtaining ZK and succinctness.

Nova's approach is substantially cheaper compared to previous work.
In particular, at each incremental step, Nova's prover performs two MSMs proportional to the size of the circuit proven.
In addition Nova's proof generation is inherently incremental, that is, it produces a proof for each step and then uses its recursion capabilities to prove the single step.
Hence, it can be more easily distributed and parallelised.

Nova does not support parallel proof generation but there is a compiler to transform constructions such as Nova to support parallel proving.
Refer to the following paper: [Recursive composition and bootstrapping for SNARKs and proof-carrying data].

This paper primarily addresses three limitations with Nova:  
- **Folding higher degree constraints**. 
    - Nova handles R1CS constraints, where each constraint is of at most degree $2$. In practice, we can have custom gates capturing constraints of higher degree. For example in Plonkish[74] we have multivariate constraint systems with high degrees. These customizable, high-degree constraint systems are often more compact than equivalent R1CS.  As a concrete example, a single iteration of MinRoot[36] can be represented with one degree-5 constraint in Plonkish.  Whereas, R1CS needs three constraints.

    -   Sangria[51] shows that Nova can be adapted to handle Plonkish but the number of cross-terms that the prover must commit to increases linearly with the degre of the constraint. The prover must incur $O(n\cdot d)$ cryptographic operations to commit to $O(d)$ cross-terms. Hence, there is no significant benefit of using Snagria with high-degree constraints, compared to using Nova with the corresponding low-degree ones. Concretely, Sangria applied to Miniroot requires 5 scalar MSM's and additional field work to compute the cross-terms, and Nova requires 6 MSMs for the corresponding R1CS for the Miniroot.

    - A key question is whether one can build a recursive argument for Plonkish, with Nova-like performance characteristics. In particular, for higher-degree constraints the performance of the prover must be independent of the degree of the constraint.

- **"A la carte" cost profile to prove machine execution.**
    - A classic approach to prove machine execution is to design a universal circuit for that captures all instructions, and then use Nova-style recursion to prove repeated invocation of this circuit on an input program and machine state. Here the size of proving the program is proportional to the size of the universal circuit which is huge.
    - The high cost means the designers try to keep the universal circuit very small by keeping the instruction set minimal. But this implies we require even simple programs require huge execution lengths.
    - An open question is whether one can achieve an “a la carte” cost profile, where the cost of proving a step of a program execution is proportional only to the size of the circuit representing the instruction invoked by the program step and independent of the circuit sizes of the uninvoked instructions.

- **providing zero-knowledge without needing zkSNARKs**

- **Efficient instantiation over a cycle of elliptic curves**
    - Nova’s approach, like in [4], requires representing a verifier (which happens to be the the non-interactive folding scheme verifier) as a circuit on both curves in the cycle of curves. For Nova,  the circuit defined over the second curve in the cycle has rougly 10, 000 multiplication gates (and more than 100,000 non-zero entries in R1CS matrices).
    - In practice, one often wants to use a “half”-pairing cycle, $E_1/E_2$, where $E_1$ has pairings but $E_2$ does not. Think of the BN254 cycle of curves with Grumpkin. The constraint system over $E_1$ can be proved succinctly using Groth 16 with the help of pairings. But for the one over $E_2$, we have to first write the circuit $C$ for its verifier over $E_1$, and then prove it using Groth16. Unfortunately the size of $C$ is at least $70 \times 10^6$.
    - An open question is whether one can substantially reduce the size of the circuit defined over the second curve in the cycle, which in turn reduces the size of $C$.

#### CycleFold notes (Section 8)

First recall the 2-cycle approach to instantiate SNARK based recursive arguments in [4]. Then describe how Nova adapts this to the context of folding-based recursive arguments.

**Two cycle approach in [4]** 

- The starting point of [4] is a pairing pased SNARK instantiated over a pairing firendly curve $E$. The proof system can prove constraint system defined over $E$' scalar field. Verifying a proof requires a handful of pairing operations which are naturally represented as operations over $E$'s base field.

- Let $\Pi_1$, $\Pi_2$ denote two SNARK schemes over the scalar fields of $E_1$, $E_2$ respectively. Naturally, proof produced by $\Pi_1$ can be efficiently verfied by a constraint system supported by $\Pi_2$ and vice versa. This is because the algroithm to verify proofs produced by $\Pi_1$ involves operations over $E_1$'s base field which by design equals the scalar field of $E_2$. In other words, teh concstraint system supported by $\Pi_2$ can efficiently encode the SNARK verifier for $\Pi_1$.

- To realize IVC at Stpe $i$ in [4], the prover proceeds as follows:
    - Using $\Pi_1$ the prover produces a SNARK proof $\pi_i^{(1)}$ that proves it has executes Step $i$ of the desired computation and has succesfully verified a SNARK proof $\pi_{i-1}^{(2)}$ from Step $i-1$.
    - Using $\Pi_2$ the prover produces a proof $\pi_i^{(2)}$ that it knows a SNARK $\pi_i^{(1)}$ and has successfully verified it.

- Note that $\pi_i^{(2)}$ is the IVC proof at the end of Step $i$. At Step $i+1$, the prove starts with $\pi_i^{(2)}$ and repeats the above procedure for the $(i+1)$-th step of the computation. A key takeaway is that this approach requires representing the SNARK verifier as a circuit on both curves in the cycle.

**Nova's instantiation over a 2-cycle of elliptic curves**

- 


